# 1. 기본 라이브러리, 데이터 가져오기

In [2]:
import pandas as pd
from glob import glob
import os
import numpy as np
from tqdm import tqdm, tqdm_notebook

import random
import torch
import torch.nn.functional as F

In [3]:
from google import colab
colab.drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
# go to file
os.chdir('/content/drive/MyDrive/dacon')

In [5]:
train = pd.read_csv("train_data.csv")
test= pd.read_csv("test_data.csv")
submission = pd.read_csv("sample_submission.csv")

#dev_data = pd.read_json("klue-nli-v1.1_dev.json")

In [6]:
import random

def seed_everything(seed:int = 1004):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything(42)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


#2. Premise Back Translate

## 01. Translate using Papago (KOR->EN)

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

In [8]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [9]:
import time

In [13]:
# Setting results path
data_path= '/content/drive/MyDrive/dacon/NPY파일/'

- 페이지가 완전히 Load되기까지 기다리가 -> Webdriver 사용

In [ ]:
def chrome_setting():
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options=chrome_options)
  return driver

In [ ]:
driver=chrome_setting()

In [ ]:
#파파고 크롤링
def kor_to_trans(text_data, trans_lang,start_index,final_index):

  target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

  for i in tqdm(range(start_index,final_index)): 
    
    if (i!=0)&(i%99==0):
      driver.implicitly_wait(2)
      print('{}th : '.format(i), backtrans)
      np.save(data_path+'kor_to_eng_train_{}_{}.npy'.format(start_index,final_index),trans_list)
    
    try:
      driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
      time.sleep(1.5)
      element=WebDriverWait(driver, 10).until(target_present)
      time.sleep(0.1)
      backtrans = element.text 

      if (backtrans=='')|(backtrans==' '):
        element=WebDriverWait(driver, 10).until(target_present)
        backtrans = element.text 
        trans_list.append(backtrans)
      else:
        trans_list.append(backtrans)
    
    except:
      trans_list.append('')

In [ ]:
trans_list=[]
kor_to_trans(train['premise'], 'en',0,2500)
np.save(data_path+'kor_to_eng_train_0_2500.npy',trans_list)

trans_list=[]
kor_to_trans(train['premise'], 'en',2500,5000)
np.save(data_path+'kor_to_eng_train_2500_5000.npy',trans_list)

trans_list=[]
kor_to_trans(train['premise'], 'en',5000,7500)
np.save(data_path+'kor_to_eng_train_5000_7500.npy',trans_list)

trans_list=[]
kor_to_trans(train['premise'], 'en',7500,10000)
np.save(data_path+'kor_to_eng_train_7500_10000.npy',trans_list)

trans_list=[]
kor_to_trans(train['premise'], 'en',10000,20000)
np.save(data_path+'kor_to_eng_train_10000_20000.npy',trans_list)

trans_list=[]
kor_to_trans(train['premise'], 'en',20000,len(train))
np.save(data_path+'kor_to_eng_train_20000_all.npy',trans_list) #24998개 번역

In [14]:
eng_data0=np.load(data_path+'kor_to_eng_train_0_2500.npy')
eng_data1=np.load(data_path+'kor_to_eng_train_2500_5000.npy')
eng_data2=np.load(data_path+'kor_to_eng_train_5000_7500.npy')
eng_data3=np.load(data_path+'kor_to_eng_train_7500_10000.npy')
eng_data4=np.load(data_path+'kor_to_eng_train_10000_20000.npy')
eng_data5=np.load(data_path+'kor_to_eng_train_20000_all.npy')

eng_data=[*eng_data0,*eng_data1,*eng_data2,*eng_data3,*eng_data4,*eng_data5]
eng_data=pd.DataFrame(eng_data,columns=['eng_premise'])

In [ ]:
back_train=pd.concat([train,eng_data],axis=1)
back_train

## 02. Googletrans 로 Retry Untranslated Sentences
- 시간 제한을 둬서인지 271개 문장이 번역X


In [ ]:
back_train[(back_train['eng_premise']=='')]

In [17]:
no_back_train = back_train[(back_train['eng_premise']=='')]

In [ ]:
!pip install googletrans==4.0.0-rc1         
import googletrans
from googletrans import Translator

In [19]:
def translate_google_en(df, col, lang='en'):
 
  translator = Translator()
  df['eng'] = [translator.translate(i, src='ko', dest=lang).text for i in df[col]]   # 한영 변환을 새로운 칼럼 'eng'에 담습니다
   
  tmp1 = df.drop(['eng'], axis=1)                                                    # 2개의 temporary 데이터프레임을 생성해 원본과 한영번역 데이터프레임을 합칩니다
  tmp2 = df.drop([col], axis=1)
  tmp2.rename(columns={'eng':'data'}, inplace=True)
  result = pd.concat([tmp1,tmp2], ignore_index=True)
  result = result.drop_duplicates()
  return result                    

In [ ]:
back_train2 = translate_google_en(no_back_train, 'premise')

In [21]:
back_train2 = back_train2[['index', 'premise', 'hypothesis', 'label', 'data' ]][271:]
back_train2 = back_train2.reset_index(drop=True)

- 인덱스 기준으로 빈 셀 채워넣기

In [ ]:
# 채워넣기
for i,j in zip(back_train2['index'], range(271)) :
  back_train['eng_premise'].iloc[i] = back_train2['data'].iloc[j]

In [ ]:
back_train

In [24]:
back_train #완성!!
train = back_train

In [ ]:
back_train.to_csv("premise_translate_eng.csv", index=False) 

## 03. Translate using Papago (EN -> KOR)

In [25]:
# 영한 파파고 번역기 크롤링
def trans_to_kor(transed_list, transed_lang,start_index,final_index): 
  
  target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

  for i in tqdm(range(start_index,final_index)): 
    
    if (i!=0)&(i%99==0):
      time.sleep(1.5)
      print('{}th : '.format(i), backtrans)
      np.save(data_path+'eng_to_kor_train_{}_{}.npy'.format(start_index,final_index),trans_list)
    
    try:
      driver.get('https://papago.naver.com/?sk=en&tk='+transed_lang+'&st='+transed_list[i])
      time.sleep(2)
      element=WebDriverWait(driver, 10).until(target_present)
      time.sleep(0.2)
      backtrans = element.text 

      if (backtrans=='')|(backtrans==' '):
        element=WebDriverWait(driver, 10).until(target_present)
        backtrans = element.text 
        trans_list.append(backtrans)
      else:
        trans_list.append(backtrans)
    
    except:
      trans_list.append('')

In [ ]:
# Setting results path
data_path= '/content/drive/MyDrive/dacon/한글번역본/'

In [ ]:
trans_list=[]
trans_to_kor(train['eng_premise'], 'ko',0,10000)
np.save(data_path+'premise_eng_to_kor_train_0_10000.npy',trans_list)

trans_list=[]
trans_to_kor(train['eng_premise'], 'ko',10000,20000)
np.save(data_path+'premise_eng_to_kor_train_10000_20000.npy',trans_list)

trans_list=[]
trans_to_kor(train['eng_premise'], 'ko',20000,len(train))
np.save(data_path+'premise_eng_to_kor_train_20000_all.npy',trans_list)

In [ ]:
back0=np.load('/content/drive/MyDrive/dacon/한글번역본/premise_eng_to_kor_train_0_10000.npy')
back1=np.load('/content/drive/MyDrive/dacon/한글번역본/premise_eng_to_kor_train_10000_20000.npy')
back2=np.load('/content/drive/MyDrive/dacon/한글번역본/premise_eng_to_kor_train_20000_all.npy')
back_train_fin=[*back0,*back1,*back2]
back_train_fin=pd.DataFrame(back_train_fin,columns=['kor_premise'])

In [ ]:
back_train_fin=pd.concat([train,back_train_fin],axis=1)
back_train_fin

- 재번역 대상 : null값을 가질 경우 & 번역 셀의 길이가 원본의 0.2 이하일 경우 => Googletrans

In [ ]:
back_train_fin[(back_train_fin['kor_premise']=='')].shape

(36, 5)

In [ ]:
# 0.2내로 너무 짧은 번역
for i in back_train_fin['kor_premise'].map(len) :
  vv = back_train_fin[(back_train_fin['kor_premise'].map(len) < i * 0.2)] #총 56개

In [ ]:
no_back_train = vv
back_train2 = translate_google_en(no_back_train, 'premise')

back_train2 = back_train2[['index', 'premise', 'hypothesis', 'label', 'kor_premise','data' ]][56:]
back_train2 = back_train2.reset_index(drop=True)

In [ ]:
# Googletrans
def translate_google_ko(df, col, lang='ko'):
 
  translator = Translator()
  df['eng'] = [translator.translate(i, src='en', dest=lang).text for i in df[col]]   # 한영 변환을 새로운 칼럼 'eng'에 담습니다
   
  tmp1 = df.drop(['eng'], axis=1)                                                    # 2개의 temporary 데이터프레임을 생성해 원본과 한영번역 데이터프레임을 합칩니다
  tmp2 = df.drop([col], axis=1)
  tmp2.rename(columns={'eng':'data2'}, inplace=True)
  result = pd.concat([tmp1,tmp2], ignore_index=True)
  result = result.drop_duplicates()
  return result    

In [ ]:
back_train3 = translate_google_ko(back_train2, 'data')

back_train3 = back_train3[['index', 'premise', 'hypothesis', 'label', 'kor_premise','data2' ]][56:]
back_train3 = back_train3.reset_index(drop=True)

In [ ]:
# 채워넣기
for i,j in zip(back_train3['index'], range(56)) :
  back_train_fin['kor_premise'].iloc[i] = back_train3['data2'].iloc[j]

In [ ]:
back_train_fin #kor_premise 완성!!

,index,premise,hypothesis,label,kor_premise
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction,씨름은 예로부터 전해 내려오는 남자들의 대표적인 놀이로 넓고 평평한 백사장이나 마당...
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction,삼성은 자작극을 벌인 2명에 대해 형사고발 등 법적 대응을 검토하고 있다고 밝혔지만...
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment,이를 위해 예측형 범죄예방시스템이 구축되고 고도화된다.
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral,광주광역시가 재개발 정비사업 원주민 종합대책을 마련하는 등 원주민 보호에 적극 나섰다.
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral,소비자와 직원들의 사랑을 받는 기업으로 정말 오래 지속하고 싶다면 이런 상황에서 책...
...,...,...,...,...,...
24993,24993,"오페라에 비하여 오라토리오에서는 독창보다도 합창이 중시되며, 테스토 또는 이스토리쿠...",오라토리오에서 테스토의 역할이 가장 중요하다.,neutral,"오페라에 비해 오라토리오에서는 독주보다 후렴구가 더 중요하며, 테스토나 이스토리쿠스..."
24994,24994,지하철역까지 걸어서 5분 정도 걸립니다.,지하철역까지 도보로 5분 정도 걸립니다.,entailment,지하철역까지 걸어서 5분 정도 걸립니다.
24995,24995,한편 이날 중앙방역대책본부는 집단 감염이 발생한 음식점 관련 역학조사 결과를 공개했다.,중악방역대책본부는 집단 감염과 관련한 모든 정보를 비공개했다.,contradiction,한편 이날 중앙질병관리본부는 집단감염이 발생한 음식점과 관련한 역학조사 결과를 발표했다.
24996,24996,마미손이 랩을 하자 시청자들은 그의 정체를 파악했다.,시청자들은 마미손의 정체를 안다.,entailment,마미손이 랩을 하자 시청자들은 마미손의 신원을 확인했다.


In [ ]:
back_train_fin.to_csv("premise_translate_final.csv", index=False) 

# 3. Hypothesis Back Translate (위 과정 반복)

## 01. Translate using Papago (KOR->EN)

In [ ]:
#크롤링
def kor_to_trans(text_data, trans_lang,start_index,final_index):

  target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

  for i in tqdm(range(start_index,final_index)): 
    
    if (i!=0)&(i%99==0):
      driver.implicitly_wait(2)
      print('{}th : '.format(i), backtrans)
      np.save(data_path+'kor_to_eng_train_{}_{}.npy'.format(start_index,final_index),trans_list)
    
    try:
      driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
      time.sleep(1.5)
      element=WebDriverWait(driver, 10).until(target_present)
      time.sleep(0.1)
      backtrans = element.text 

      if (backtrans=='')|(backtrans==' '):
        element=WebDriverWait(driver, 10).until(target_present)
        backtrans = element.text 
        trans_list.append(backtrans)
      else:
        trans_list.append(backtrans)
    
    except:
      trans_list.append('')

In [ ]:
trans_list=[]
kor_to_trans(train['hypothesis'], 'en',0,2500)
np.save(data_path+'kor_to_eng_train_0_2500.npy',trans_list)

trans_list=[]
kor_to_trans(train['hypothesis'], 'en',2500,5000)
np.save(data_path+'kor_to_eng_train_2500_5000.npy',trans_list)

trans_list=[]
kor_to_trans(train['hypothesis'], 'en',5000,7500)
np.save(data_path+'kor_to_eng_train_5000_7500.npy',trans_list)

trans_list=[]
kor_to_trans(train['hypothesis'], 'en',7500,10000)
np.save(data_path+'kor_to_eng_train_7500_10000.npy',trans_list)

trans_list=[]
kor_to_trans(train['hypothesis'], 'en',10000,12500)
np.save(data_path+'kor_to_eng_train_10000_12500.npy',trans_list)

trans_list=[]
kor_to_trans(train['hypothesis'], 'en',12500,15000)
np.save(data_path+'kor_to_eng_train_12500_15000.npy',trans_list)

trans_list=[]
kor_to_trans(train['hypothesis'], 'en',15000,17500)
np.save(data_path+'kor_to_eng_train_15000_17500.npy',trans_list)

trans_list=[]
kor_to_trans(train['hypothesis'], 'en',17500,20000)
np.save(data_path+'kor_to_eng_train_17500_20000.npy',trans_list)

trans_list=[]
kor_to_trans(train['hypothesis'], 'en',20000,len(train))
np.save(data_path+'kor_to_eng_train_20000_all.npy',trans_list) #24998개 번역

In [ ]:
eng_data0=np.load(data_path+'kor_to_eng_train_0_2500.npy')
eng_data1=np.load(data_path+'kor_to_eng_train_2500_5000.npy')
eng_data2=np.load(data_path+'kor_to_eng_train_5000_7500.npy')
eng_data3=np.load(data_path+'kor_to_eng_train_7500_10000.npy')
eng_data4=np.load(data_path+'kor_to_eng_train_10000_12500.npy')
eng_data5=np.load(data_path+'kor_to_eng_train_12500_15000.npy')
eng_data6=np.load(data_path+'kor_to_eng_train_15000_17500.npy')
eng_data7=np.load(data_path+'kor_to_eng_train_17500_20000.npy')
eng_data8=np.load(data_path+'kor_to_eng_train_20000_all.npy')

eng_data=[*eng_data0,*eng_data1,*eng_data2,*eng_data3,*eng_data4,*eng_data5,*eng_data6,*eng_data7,*eng_data8]
eng_data=pd.DataFrame(eng_data,columns=['eng_hypothesis'])

In [ ]:
back_train=pd.concat([train,eng_data],axis=1)
back_train

## 02. Googletrans 로 Retry Untranslated Sentences
- 시간 제한을 둬서인지 207개 문장이 번역X

In [ ]:
no_back_train = back_train[(back_train['eng_hypothesis']=='')]

In [ ]:
!pip install googletrans==4.0.0-rc1         
import googletrans
from googletrans import Translator

In [ ]:
back_train2 = translate_google_en(no_back_train, 'hypothesis')

back_train2 = back_train2[['index', 'premise', 'hypothesis', 'label', 'data' ]][207:]
back_train2 = back_train2.reset_index(drop=True)

- 인덱스 기준으로 빈 셀 채워넣기

In [ ]:
# 채워넣기
for i,j in zip(back_train2['index'], range(207)) :
  back_train['eng_hypothesis'].iloc[i] = back_train2['data'].iloc[j]

In [ ]:
back_train #완성!!
train = back_train

## 03. Translate using Papago (EN -> KOR)

In [ ]:
trans_list=[]
trans_to_kor(train['eng_hypothesis'], 'ko',0,10000)
np.save(data_path+'premise_eng_to_kor_train_0_10000.npy',trans_list)

trans_list=[]
trans_to_kor(train['eng_hypothesis'], 'ko',10000,20000)
np.save(data_path+'premise_eng_to_kor_train_10000_20000.npy',trans_list)

trans_list=[]
trans_to_kor(train['eng_hypothesis'], 'ko',20000,len(train))
np.save(data_path+'premise_eng_to_kor_train_20000_all.npy',trans_list)

In [ ]:
# Setting results path
data_path= '/content/drive/MyDrive/dacon/NPY파일'

In [ ]:
back0=np.load('/content/drive/MyDrive/dacon/NPY파일/eng_to_kor_train_0_10000.npy')
back1=np.load('/content/drive/MyDrive/dacon/NPY파일/eng_to_kor_train_10000_20000.npy')
back2=np.load('/content/drive/MyDrive/dacon/NPY파일/eng_to_kor_train_20000_all.npy')
back_train_fin=[*back0,*back1,*back2]
back_train_fin=pd.DataFrame(back_train_fin,columns=['kor_hypothesis'])

In [ ]:
back_train_fin=pd.concat([train,back_train_fin],axis=1)
back_train_fin

- 재번역 대상 : null값을 가질 경우 & 번역 셀의 길이가 원본의 0.2 이하일 경우 => Googletrans

In [ ]:
back_train_fin[(back_train_fin['kor_hypothesis']=='')].shape

(374, 5)

In [ ]:
# 0.2내로 너무 짧은 번역
for i in back_train_fin['kor_hypothesis'].map(len) :
  vv = back_train_fin[(back_train_fin['kor_hypothesis'].map(len) < i * 0.2)] #392개

In [ ]:
no_back_train = vv

back_train2 = translate_google_en(no_back_train, 'hypothesis')
back_train2 = back_train2[['index', 'premise', 'hypothesis', 'label', 'kor_hypothesis','data' ]][392:]
back_train2 = back_train2.reset_index(drop=True)

In [ ]:
back_train3 = translate_google_ko(back_train2, 'data')
back_train3 = back_train3[['index', 'premise', 'hypothesis', 'label', 'kor_hypothesis','data2' ]][392:]
back_train3 = back_train3.reset_index(drop=True)

In [ ]:
# 채워넣기
for i,j in zip(back_train3['index'], range(392)) :
  back_train_fin['kor_hypothesis'].iloc[i] = back_train3['data2'].iloc[j]

In [ ]:
back_train_fin #완성!!

,index,premise,hypothesis,label,kor_hypothesis
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction,씨름에 나오는 여자들의 놀이입니다.
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction,그들 자신의 연극을 만든 세 사람이 있었다.
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment,예측형 범죄예방 시스템 구축과 고도화가 목적이기 때문이다.
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral,그 원주민들은 포괄적인 조치에 만족했습니다.
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral,이런 상황에서 책임감을 드러내는 기업은 극소수다.
...,...,...,...,...,...
24993,24993,"오페라에 비하여 오라토리오에서는 독창보다도 합창이 중시되며, 테스토 또는 이스토리쿠...",오라토리오에서 테스토의 역할이 가장 중요하다.,neutral,오라토리오에서 테스토의 역할이 가장 중요하다.
24994,24994,지하철역까지 걸어서 5분 정도 걸립니다.,지하철역까지 도보로 5분 정도 걸립니다.,entailment,지하철역까지 걸어서 5분 정도 걸립니다.
24995,24995,한편 이날 중앙방역대책본부는 집단 감염이 발생한 음식점 관련 역학조사 결과를 공개했다.,중악방역대책본부는 집단 감염과 관련한 모든 정보를 비공개했다.,contradiction,본사 본부는 집단 감염에 관한 모든 정보를 촉진해야한다.
24996,24996,마미손이 랩을 하자 시청자들은 그의 정체를 파악했다.,시청자들은 마미손의 정체를 안다.,entailment,마미손이 누군지 아시네요.


In [ ]:
back_train_fin.to_csv("hypothesis_translate_final.csv", index=False) 

# 4. DEV 추가데이터 Back Translate (위 과정 반복)
- 위 과정의 반복이므로 코드가 동일해 따로 데이터파일을 첨부하겠습니다.  